In [1]:
import os
import numpy as np

from hmog import HmogHelper
from tqdm import tqdm

dataset_path = 'hmog_public_dataset/'

subjects = HmogHelper.list_folder(dataset_path)

session_maps = {}

for subject in subjects:
    ww_subject_sessions = HmogHelper.filter_user_sessions_by_type(os.path.join(dataset_path, subject), HmogHelper.WW_SESSION_TYPES)
    ws_subject_sessions = HmogHelper.filter_user_sessions_by_type(os.path.join(dataset_path, subject), HmogHelper.WS_SESSION_TYPES)

    session_maps[subject] = {'w': [],
                             's': []}
    for session in ww_subject_sessions:
        session_maps[subject]['w'].append(HmogHelper.read_person_session(os.path.join(dataset_path, subject, session)))
        HmogHelper.preprocess_session_data(session_maps[subject]['w'][-1])
    for session in ws_subject_sessions:
        session_maps[subject]['s'].append(HmogHelper.read_person_session(os.path.join(dataset_path, subject, session)))
        HmogHelper.preprocess_session_data(session_maps[subject]['s'][-1])

In [2]:
session_hmog_matrices = {}

excluded_subjects = []

for subject in tqdm(session_maps):
    sessions_w = session_maps[subject]['w']
    sessions_s = session_maps[subject]['s']

    assert len(sessions_w) > 3
    assert len(sessions_s) > 3

    success_w1, w1 = HmogHelper.extract_hmog_features(sessions_w[0], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_w2, w2 = HmogHelper.extract_hmog_features(sessions_w[1], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_w3, w3 = HmogHelper.extract_hmog_features(sessions_w[2], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_w4, w4 = HmogHelper.extract_hmog_features(sessions_w[3], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    success_s1, s1 = HmogHelper.extract_hmog_features(sessions_s[0], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_s2, s2 = HmogHelper.extract_hmog_features(sessions_s[1], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_s3, s3 = HmogHelper.extract_hmog_features(sessions_s[2], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_s4, s4 = HmogHelper.extract_hmog_features(sessions_s[3], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    extract_success = success_w1 and success_w2 and success_w3 and success_w4 and success_s1 and success_s2 and success_s3 and success_s4

    if extract_success:
        session_hmog_matrices[subject] = {'w': [],
                                          's': []}

        session_hmog_matrices[subject]['w'] = []
        session_hmog_matrices[subject]['w'].append(w1)
        session_hmog_matrices[subject]['w'].append(w2)
        session_hmog_matrices[subject]['w'].append(w3)
        session_hmog_matrices[subject]['w'].append(w4)

        session_hmog_matrices[subject]['s'] = []
        session_hmog_matrices[subject]['s'].append(s1)
        session_hmog_matrices[subject]['s'].append(s2)
        session_hmog_matrices[subject]['s'].append(s3)
        session_hmog_matrices[subject]['s'].append(s4)
    else:
        excluded_subjects.append(subject)

print(f'excluded subjects: {excluded_subjects}')
# std_deviations, session_hmog_vector = calc_std_and_scale(session_hmog_vector)

100%|██████████| 100/100 [1:29:41<00:00, 53.82s/it]

excluded subjects: ['733162']


In [3]:
import pickle

pickled_ds_filename = 'hmog_dataset_ww_ws'

if not os.path.exists(pickled_ds_filename):
    hmog_dataset_file = open(pickled_ds_filename, 'wb')
    pickle.dump(session_hmog_matrices, hmog_dataset_file)
    hmog_dataset_file.close()